In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

def dark_channel(image, size=15):
    """Calculate the dark channel prior of an image."""
    min_channel = np.min(image, axis=2)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (size, size))
    dark_channel = cv2.erode(min_channel, kernel)
    return dark_channel

def estimate_atmospheric_light(image, dark_channel):
    """Estimate atmospheric light based on the dark channel."""
    h, w = dark_channel.shape
    num_pixels = h * w
    num_brightest_pixels = int(max(num_pixels // 1000, 1))  # Top 0.1% brightest pixels
    indices = np.unravel_index(np.argsort(dark_channel.ravel())[-num_brightest_pixels:], dark_channel.shape)
    A = np.mean(image[indices], axis=0)
    return A

In [ ]:
def defog_image(image):
    """Defog the input image using improved dark channel prior."""
    I = image.astype(float)
    dark_channel_img = dark_channel(I)
    A = estimate_atmospheric_light(I, dark_channel_img)
    omega = 0.95
    t = 1 - omega * (dark_channel_img / np.max(A))  # Normalize by max atmospheric light value
    t = np.clip(t, 0.1, 1)  # Avoid division by zero
    t = cv2.bilateralFilter(t.astype(np.float32), 5, 0.1, 0.1)
    J = np.empty_like(I)
    for i in range(3):  # For each color channel
        J[:, :, i] = (I[:, :, i] - A[i]) / t + A[i]
    J = np.clip(J, 0, 255).astype(np.uint8)  # Ensure pixel values are valid
    return J

In [ ]:
def detect_lines(cropped_edges, image):
    """Detect lines in the cropped edges image and return the line image."""
    lines = cv2.HoughLinesP(cropped_edges, rho=1, theta=np.pi / 180, threshold=50, minLineLength=100, maxLineGap=50)

    # Create an image to draw the lines on
    line_image = np.zeros_like(image)

    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            # Calculate the slope of the line
            if x2 != x1:  # Avoid division by zero for vertical lines
                slope = (y2 - y1) / (x2 - x1)
            else:
                slope = np.inf  # Assign infinite slope for vertical lines

            # Filter out vertical lines (considered vertical if slope is steep)
            if abs(slope) > 0.5:  # Adjust this threshold to filter out nearly vertical lines
                cv2.line(line_image, (x1, y1), (x2, y2), (0, 255, 0), 5)  # Draw lane lines

    return line_image

In [ ]:
def detect_lanes(image):
    """Detect lanes in a defogged image while filtering out vertical lines."""
    # Convert image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    cv2_imshow(gray)  # Show grayscale image

    # Apply Gaussian blur to reduce noise
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    cv2_imshow(blur)  # Show blurred image

    # Edge detection using Canny
    edges = cv2.Canny(blur, 50, 150)
    cv2_imshow(edges)  # Show edges

    # Define a refined region of interest (ROI) mask to focus on the lane area
    height, width = edges.shape
    mask = np.zeros_like(edges)
    polygon = np.array([[  # Define the polygon to mask the region of interest
        (0, height),
        (width, height),
        (width, height // 1.7),  # Higher point to focus more on the lanes
        (0, height // 1.7)
    ]], np.int32)
    cv2.fillPoly(mask, polygon, 255)
    cropped_edges = cv2.bitwise_and(edges, mask)
    cv2_imshow(cropped_edges)  # Show cropped edges

    # Call detect_lines to get the line image
    line_image = detect_lines(cropped_edges, image)

    # Combine line image with the original image
    lane_image = cv2.addWeighted(image, 0.8, line_image, 1, 1)

    return lane_image

In [ ]:
foggy_image = cv2.imread("/images.png")

cv2_imshow(foggy_image)
# Step 1: Defog the image
defogged_image = defog_image(foggy_image)
cv2_imshow(defogged_image)  # Show defogged image

# Step 2: Detect lanes on the defogged image
lane_image = detect_lanes(defogged_image)

# Display the final result
cv2_imshow(lane_image)

In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

def add_haze(image, haze_intensity=0.7, light_intensity=0.8):
    """
    Add synthetic haze to an image.

    Parameters:
        image (np.array): Input clear image (BGR format).
        haze_intensity (float): Intensity of the haze (0 to 1).
        light_intensity (float): Intensity of atmospheric light (0 to 1).

    Returns:
        hazy_image (np.array): Hazy image.
    """
    h, w, c = image.shape

    # Create a uniform transmission map to simulate haze
    transmission_map = np.full((h, w), 1 - haze_intensity, dtype=np.float32)
    transmission_map = transmission_map[:, :, np.newaxis]  # Expand dimensions for broadcasting

    # Atmospheric light
    A = np.full((1, 1, c), light_intensity * 255, dtype=np.float32)

    # Apply haze model
    hazy_image = image.astype(np.float32) * transmission_map + A * (1 - transmission_map)
    hazy_image = np.clip(hazy_image, 0, 255).astype(np.uint8)

    return hazy_image

# Load an image
image = cv2.imread('/19.jpg')  # Replace with your image path

# Add haze
hazy_image = add_haze(image, haze_intensity=0.89, light_intensity=0.9)

# Display the images
cv2_imshow(image)
cv2_imshow(hazy_image)

# Wait for a key press and close the windows
cv2.waitKey(0)
cv2.destroyAllWindows()